# Extract variant carriers and perform annotation using GP2 NBA data (PLINK files from Release 9)

## Exploring the Global Landscape of Rare Causal and Common High-Risk Variants in Parkinson’s Disease

`GP2 ❤️ Open Science 😍`

## Description:

This notebook contains the code and workflow used in the study: **“Exploring the Global Landscape of Rare Causal and Common High-Risk Variants in Parkinson’s Disease”**.

In this notebook we extract variant carriers and perform annotation using GP2 NBA data (PLINK files from Release 9).

### Outline:

* **0. Set Up**

* **1. Install software and define paths**
    * 1.1. Install plink
    * 1.2. Install bcftools
    * 1.3. Install ANNOVAR
    * 1.4 Create working directory and set paths

* **2. Create and edit .range file with genes of interest and genomic coordinates**

* **3. Prepare genotyping files**
    * 3.1 Template to get chromosomal positions for NBA marker IDs
    * 3.2 Convert NBA marker IDs from genotyping files into chromosomal positions

* **4. Extract variants in genes of interest and annotation**
    * AAC (as an example)

## 0. Set Up

In [ ]:
## Use the os package to interact with the environment
import os

## Bring in Pandas for Dataframe functionality
import pandas as pd

import subprocess

## Numpy for basics
import numpy as np

## Use pathlib for file path manipulation
import pathlib

## Use StringIO for working with file contents
from io import StringIO

## Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

## Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

## Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

## BigQuery for querying data
from google.cloud import bigquery

## Import Sys
import sys as sys

## 1. Install software and define paths

### 1.1. Install plink

In [ ]:
%%capture
%%bash

# Install plink 1.9
mkdir -p ~/tools
cd ~/tools/
if test -e ~/tools/plink; then
    echo "Plink is already installed"
else
    echo "Plink is not installed"
    cd ~/tools/

    wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 

    unzip -o plink_linux_x86_64_20190304.zip
    mv plink plink1.9
fi

In [ ]:
%%bash

# chmod plink 1.9 to make sure you have permission to run the program
chmod u+x ~/tools/plink1.9

In [ ]:
%%capture
%%bash

# Install plink 2.0
cd ~/tools/
if test -e ~/tools/plink2; then
    echo "Plink2 is already installed"
else
    echo "Plink2 is not installed"
    cd ~/tools/

    wget http://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip

    unzip -o plink2_linux_x86_64_latest.zip
fi

In [ ]:
%%bash

# chmod plink 2 to make sure you have permission to run the program
chmod u+x ~/tools/plink2

### 1.2 Install bcftools

In [ ]:
%%capture
%%bash 

# Install bcftools
cd /home/jupyter/tools/

if test -e /home/jupyter/tools/bcftools; then
    echo "bcftools is already installed in /home/jupyter/tools/"
else
    echo -e "Downloading bcftools \n    -------"
    git clone --recurse-submodules https://github.com/samtools/htslib.git
    git clone https://github.com/samtools/bcftools.git
    cd bcftools
    make
    echo -e "\n bcftools downloaded and unzipped in /home/jupyter/tools \n "

fi

In [ ]:
%%bash

# chmod bcftools to make sure you have permission to run the program
chmod +x /home/jupyter/tools/bcftools

### 1.3 Install ANNOVAR

In [ ]:
%%capture
%%bash

# Install ANNOVAR: We are adding the download link after registration on the annovar website
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e ~/tools/annovar ; then
    echo "annovar is already installed in /home/jupyter/workspace/ws_files/fangz_workdir/annovar"
else
    echo "annovar is not installed"
    cd ~/tools

    wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

    tar xvfz annovar.latest.tar.gz

fi

In [ ]:
%%capture
%%bash

# Install ANNOVAR: Download resources for annotation

cd ~/tools/annovar
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20240917 humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp47a humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad41_genome humandb/


### 1.4 Create working directory and set paths

In [ ]:
# Create a folder on your workspace
print("Making a working directory")
!mkdir -p /home/jupyter/workspace/ws_files/your_directory
!mkdir -p /home/jupyter/workspace/ws_files/your_directory/bed_files
!mkdir -p /home/jupyter/workspace/ws_files/your_directory/temp_files
!mkdir -p /home/jupyter/workspace/ws_files/your_directory/results

workdir="/home/jupyter/workspace/ws_files/your_directory"

## 2. Create and edit .range file with genes of interest and genomic coordinates

#### Create a .txt file with the gene information (include all genes in which you want to extract variant carriers)
Use the following format: ```CHR START END GENE```

**CHR** refers to the chromosome the gene is located on (e.g., chr1)

**START** refers to the chromosomal position at which the gene starts

**END** refers to the chromosomal position at which the gene ends

**GENE** refers to the gene name (optional)

Use the ensembl genome browser to obtain these information (https://useast.ensembl.org/index.html). 

In [ ]:
# Convert the .txt into a .range file
input_file = "/home/jupyter/workspace/ws_files/your_directory/bed_files/chrom_pos.txt"
output_file = "/home/jupyter/workspace/ws_files/your_directory/bed_files/chrom_pos.range"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        cleaned_line = "\t".join(line.strip().split())  # Replace spaces with actual tabs
        outfile.write(cleaned_line + "\n")

print("Conversion complete! File saved as:", output_file)

In [ ]:
# Read in the .range file
range = pd.read_csv('/home/jupyter/workspace/ws_files/your_directory/bed_files/chrom_pos.range', sep="\t", header=None, dtype=str)
range

## 3. Prepare genotyping files

#### 3.1 Template to get chromosomal positions for NBA marker IDs

In [ ]:
! wget "https://hgdownload.soe.ucsc.edu/goldenpath/hg38/bigZips/hg38.fa.gz" 

In [ ]:
!mv /home/jupyter/workspace/ws_files/your_directory/hg38.fa.gz /home/jupyter/workspace/ws_files/your_directory/docs/hg38.fa.gz

#### 3.2 Convert NBA marker IDs from genotyping files into chromosomal positions

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/EUR/EUR_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/EUR_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/AAC/AAC_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/AAC_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/AFR/AFR_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/AFR_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/AJ/AJ_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/AJ_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/AMR/AMR_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/AMR_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/CAH/CAH_release9_flipped_vwb \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/CAH_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/CAS/CAS_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/CAS_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/EAS/EAS_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/EAS_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/FIN/FIN_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/FIN_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/MDE/MDE_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/MDE_release9_new

In [ ]:
! /home/jupyter/tools/plink2 --pfile /home/jupyter/workspace/path/to/release9/raw_genotypes_flipped/SAS/SAS_release9_flipped_vwb \
--fa /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/hg38.fa.gz \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/workspace/ws_files/GP2_R9_monogenic/docs/SAS_release9_new

## 4. Extract variants in genes of interest and annotation

### AAC (as an example)

In [ ]:
%%bash

# Define all the variables
INPUT_PLINK_DIR="/home/jupyter/workspace/ws_files/your_directory/docs"
PLINK2_PATH="/home/jupyter/tools/plink2"
TEMP_DIR="/home/jupyter/workspace/ws_files/your_directory/temp_files"
ANCESTRY="AAC"

# Run PLINK2
$PLINK2_PATH --pfile "${INPUT_PLINK_DIR}/${ANCESTRY}_release9_new" \
    --mac 1 \
    --extract range /home/jupyter/workspace/ws_files/your_directory/bed_files/chrom_pos.range \
    --make-pgen \
    --out "${TEMP_DIR}/${ANCESTRY}/${ANCESTRY}_filtered"


In [ ]:
%%bash

PLINK2_PATH="/home/jupyter/tools/plink2"
TEMP_DIR="/home/jupyter/workspace/ws_files/your_directory/temp_files"
ANCESTRY="AAC"


$PLINK2_PATH --pfile "/home/jupyter/workspace/ws_files/your_directory/temp_files/AAC/AAC_filtered" \
       --recode vcf \
       --out "${TEMP_DIR}/${ANCESTRY}/${ANCESTRY}"

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/your_directory/temp_files"
ANCESTRY="AAC"

bgzip -c ${TEMP_DIR}/${ANCESTRY}/${ANCESTRY}.vcf > ${TEMP_DIR}/${ANCESTRY}/annovar_input_${ANCESTRY}.vcf.gz
tabix -p vcf ${TEMP_DIR}/${ANCESTRY}/annovar_input_${ANCESTRY}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/your_directory"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/AAC/annovar_input_AAC.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/nba_final_AAC.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/your_directory"

head ${workdir}/results/nba_final_AAC.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/your_directory"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/nba_final_AAC.annovar.hg38_multianno.txt',sep='\t',dtype=str)

# Find variant ID column
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# Select the columns to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# Subset the columns
AAC_df=anno[all_cols_to_keep]

# Rename column
AAC_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

AAC_df

In [ ]:
# Count occurrences of each value
value_counts = AAC_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### 4.1 Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_AAC_df = AAC_df[AAC_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_AAC_df.head())


In [ ]:
# Filter out synonymous SNVs
filtered_AAC_df = filtered_AAC_df[filtered_AAC_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_AAC_df.head())

In [ ]:
# Save the filtered output
filtered_AAC_df.to_csv(f"{workdir}/results/filtered_multianno_AAC.tsv", sep="\t", index=False)

# Write out 'var_id' to extract from plink files
filtered_AAC_df['var_id'].to_csv(f"{workdir}/results/AAC_var_to_extract.txt",index=False,header=False)


#### 4.2 Extract carrier IDs and genotypes

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/your_directory"
TEMP_DIR="/home/jupyter/workspace/ws_files/your_directory/temp_files"
ANCESTRY="AAC"

~/tools/plink2 --pfile ${TEMP_DIR}/${ANCESTRY}/${ANCESTRY}_filtered \
               --extract ${workdir}/results/${ANCESTRY}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${ANCESTRY}/${ANCESTRY}_geno
               

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/your_directory/temp_files"
ANCESTRY="AAC"

aac_var = pd.read_csv(f'{TEMP_DIR}/{ANCESTRY}/{ANCESTRY}_geno.raw', sep='\s+')
aac_var

In [ ]:
# Transpose the dataframe to be row as variants and columns as samples
var_col=aac_var.columns[6:len(aac_var)]
d = aac_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=aac_var[['IID','PHENOTYPE']]

# Filtering rows where any value in 'var_col' is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
# Strip the last '_${ref_allele}', so we can keep the same variant ID as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# Store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]



In [ ]:
# Check hom as example what to expect
hom

In [ ]:
# Split the carrier ID from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# Keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
# Merge with annotation
out_hom = pd.merge(hom,filtered_AAC_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

# Rename column
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
# Repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
# Merge with annotation
out_het = pd.merge(het,filtered_AAC_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

# Rename column
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het


In [ ]:
# Check if there's any comphet by grouping gene and sample ID
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)


In [ ]:
# Write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'


In [ ]:
# Get everything and write out
merged_df_AAC = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_AAC.to_csv(f"{workdir}/results/results_AAC.tsv",sep='\t',index=False)

### Now repeat the same steps for all other ancestries (AFR, AJ, AMR, CAH, CAS, EAS, EUR, FIN, MDE, SAS)!